In [1]:
import numpy as np
import pandas as pd
from main import create_model_by_experiment_path_and_stage
import torch
import os
import netCDF4 as nc

os.environ['CUDA_VISIBLE_DEVICES']=""
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [10]:

input_data_folder = '/wk1/bcw817/data/Himawari-8/data_ready/Pacific/'
label_data_folder = '/wk1/bcw817/data/Himawari-8/data_ready_qperr/Pacific/'
data_channel = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]
print('data preparing...')

def remove_outlier_and_nan(numpy_array, upper_bound=1000):
    numpy_array = np.nan_to_num(numpy_array, copy=False)
    # numpy_array[numpy_array > upper_bound] = 0
    return numpy_array
# load and combine data
combine_time_data  = []
test_year = ['2023']
for year_i in test_year:
    input_files = os.listdir(input_data_folder+year_i)
    input_files.sort()
    for file in input_files:
        input_tmp = nc.Dataset(input_data_folder+year_i+'/'+file)
        time_tmp = input_tmp.variables['time'][:]+np.int_(file[2:-3])*1000
        time_tmp = list(map(str, time_tmp))
        input_data = input_tmp.variables['Himawari_images'][:]
        label_tmp = nc.Dataset(label_data_folder+year_i+'/'+file)
        label_data = label_tmp.variables['qperr'][:]
        if year_i == test_year[0] and file == input_files[0]:
            combine_input_data = input_data
            combine_label_data = label_data
        else:
            combine_input_data = np.concatenate((combine_input_data, input_data), axis=0)
            combine_label_data = np.concatenate((combine_label_data, label_data), axis=0)
        combine_time_data = combine_time_data+time_tmp
    print(f'finish {year_i}------------------')
print('finish data loading-----------------')
data_channel = [data_channel_index-1 for data_channel_index in data_channel]
combine_input_data = remove_outlier_and_nan(combine_input_data)[:,:-1,:,:][:,:,:-1,:][:,:,:,data_channel]
combine_label_data = remove_outlier_and_nan(combine_label_data)[:,:-1,:,:][:,:,:-1,:]

print(f"test year: {test_year} numbers: {len(combine_time_data)}")
combine_time_data = list(map(float, combine_time_data))
combine_time_data = list(map(int, combine_time_data))

# size:［B, C, H, W］
input_data_test  = np.swapaxes(np.swapaxes(combine_input_data,2,3),1,2)
label_data_test  = np.swapaxes(np.swapaxes(combine_label_data,2,3),1,2)


data preparing...
finish 2023------------------
finish data loading-----------------
test year: ['2023'] numbers: 2952


In [13]:
combine_input_data.shape

In [14]:
# predict
experiment_path = '/wk171/yungyun/for_gitea_publish/EC_torch_unet_publish/experiments/downscaling_unet_test.yml'
sub_exp_name = 'unet_try1'
model_name, weight_path = create_model_by_experiment_path_and_stage(experiment_path, sub_exp_name)
new_input_data_test = torch.from_numpy(input_data_test.astype('float32'))

# load on gpu or cpu
new_input_data_test = new_input_data_test.to(device)
model_name = model_name.to(device)
model_name.load_state_dict(torch.load(weight_path, map_location=device))
pred_rainfall = label_data_test*0.0

# evaluate
model_name.eval()
for i in range(len(pred_rainfall[:,0,0,0])):
    pred_rainfall[i,0,:,:] = model_name(new_input_data_test[i:i+1,:,:,:])[0,0,:,:].cpu().detach().numpy()

In [15]:
pred_rainfall.shape

(2952, 1, 100, 100)

In [37]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Thu Nov  4 15:44:58 2021

@author: yungyun
"""


#%% plot
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

coast = pd.read_csv('/wk171/yungyun/AI_global_model/Pangu-Weather-for-copy/plot/coast.csv')


#%% rain setting
nws_precip_colors = [
    "#fdfdfd",  # 0.01 - 0.10 inches
    "#c9c9c9",  # 0.10 - 0.25 inches
    "#9dfeff",
    "#01d2fd",  # 0.25 - 0.50 inches
    "#00a5fe",  # 0.50 - 0.75 inches
    "#0177fd",  # 0.75 - 1.00 inches
    "#27a31b",  # 1.00 - 1.50 inches
    "#00fa2f",  # 1.50 - 2.00 inches
    "#fffe33",  # 2.00 - 2.50 inches
    "#ffd328",  # 2.50 - 3.00 inches
    "#ffa71f",  # 3.00 - 4.00 inches
    "#ff2b06",
    "#da2304",  # 4.00 - 5.00 inches
    "#aa1801",  # 5.00 - 6.00 inches
    "#ab1fa2",  # 6.00 - 8.00 inches
    "#db2dd2",  # 8.00 - 10.00 inches
    "#ff38fb",  # 10.00+
    "#ffd5fd"]


precip_colormap = mpl.colors.ListedColormap(nws_precip_colors)
item = 18
clevel = [0, 0.5, 1, 2, 6, 10, 15, 20,  30, 40, 50,
          70, 90, 110, 130,150,200,300,400]
norm = mpl.colors.BoundaryNorm(clevel, item)

# for case_i in range(len(test_index)):
for case_i in range(2):
        # case_i = 20
        input_data = input_data_test[case_i,13,:,:]
        preds_data = pred_rainfall[case_i,0,:,:]
        label_data = label_data_test[case_i,0,:,:]
        
        
        font = {'family'     : 'DejaVu Sans Mono',
                'weight'     : 'bold',
                'size'       : 18
                }
        axes = {'titlesize'  : 18,
                'titleweight': 'heavy',
                'labelsize'  : 18,
                'labelweight': 'bold'
                }
        mpl.rc('font', **font)  # pass in the font dict as kwargs
        mpl.rc('axes', **axes)
        #ax.add_geometries(adm1_shapes, ccrs.PlateCarree(),
        #                  edgecolor='black', facecolor='gray', alpha=0.5)
        # Pracific
        latStart = 23; latEnd =24.2375;#25.35
        lonStart = 121.875; lonEnd = 123.1125
        lat = np.linspace(latStart,latEnd,100)
        lon = np.linspace(lonStart,lonEnd,100)
        lons, lats = np.meshgrid(lon, lat)    
        # lons, lats = np.meshgrid(EC_lon, EC_lat)
        plt.figure(figsize=(18,6))
        plt.suptitle(combine_time_data[case_i])
        plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei'] 

        # ax1 = plt.axes(projection=ccrs.PlateCarree())

        ax1 = plt.subplot(1, 3, 1)
        filled_c = ax1.contourf(lons, lats, input_data, levels=np.linspace(200,300,21),cmap='gray',linesytles=None)

        ax1.plot(coast.lon_map, coast.lat_map, color='k', linewidth=0.7)
        ax1.set_xticks(np.arange(120, 123))
        ax1.set_yticks(np.arange(22, 26))
        plt.xlim([118, 124])
        plt.ylim([21, 26])
        ax1.tick_params('both', labelsize=16)

        ax2 = plt.subplot(1, 3, 2)
        filled_c = ax2.contourf(lons, lats, preds_data, levels=clevel, norm=norm,cmap=precip_colormap,linesytles=None)
        ax2.plot(coast.lon_map, coast.lat_map, color='k', linewidth=0.7)
        ax2.set_xticks(np.arange(120, 123))
        ax2.set_yticks(np.arange(22, 26))
        plt.xlim([118, 124])
        plt.ylim([21, 26])
        ax2.tick_params('both', labelsize=16)
        ax2.tick_params('both', labelsize=16)



        ax3 = plt.subplot(1, 3, 3)
        filled_c = ax3.contourf(lons, lats, label_data, levels=clevel, norm=norm,cmap=precip_colormap,linesytles=None)
        ax3.plot(coast.lon_map, coast.lat_map, color='k', linewidth=0.7)
        ax3.set_xticks(np.arange(120, 123))
        ax3.set_yticks(np.arange(22, 26))
        plt.xlim([118, 124])
        plt.ylim([21, 26])
        ax3.tick_params('both', labelsize=16)
        ax3.tick_params('both', labelsize=16)
        plt.savefig('plt_test/'+str(combine_time_data[case_i])+'.png')
        plt.close()

/tmp/user/1004/ipykernel_15368/343249998.py:82: UserWarning: The following kwargs were not used by contour: 'linesytles'
  filled_c = ax1.contourf(lons, lats, input_data, levels=np.linspace(200,300,21),cmap='gray',linesytles=None)
/tmp/user/1004/ipykernel_15368/343249998.py:92: UserWarning: The following kwargs were not used by contour: 'linesytles'
  filled_c = ax2.contourf(lons, lats, preds_data, levels=clevel, norm=norm,cmap=precip_colormap,linesytles=None)
/tmp/user/1004/ipykernel_15368/343249998.py:104: UserWarning: The following kwargs were not used by contour: 'linesytles'
  filled_c = ax3.contourf(lons, lats, label_data, levels=clevel, norm=norm,cmap=precip_colormap,linesytles=None)


146.75

In [ ]:
plot_windy_predict

(319, 1, 112, 104)